In [ ]:
!python -m jupytools export -nb "07_resnets.ipynb" -o .

In [ ]:
%reload_ext autoreload
%autoreload 2

## Imports 

In [ ]:
#export
from collections import OrderedDict
import json
import os
from os.path import dirname, join
from functools import reduce
from pdb import set_trace

import cv2 as cv
import jupytools
import jupytools.syspath
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pretrainedmodels
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from visdom import Visdom

jupytools.syspath.add(join(dirname(os.getcwd()), 'protein_project'))
jupytools.syspath.add('rxrx1-utils')
if jupytools.is_notebook():
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm as tdqm
    
import contextlib, io
with contextlib.redirect_stderr(io.StringIO()):
    from basedir import ROOT, NUM_CLASSES
    from catalyst_visdom import BatchMetricsPlotCallback, EpochMetricsPlotCallback
    from dataset import load_data, build_stats_index, RxRxDataset
    from model.utils import freeze_all, unfreeze_layers
    from visual import rgb, six, show_1, show

In [ ]:
#export
torch.set_default_tensor_type(torch.FloatTensor)

## Dataset Preparation

### JSON

In [ ]:
#export
data_dict = load_data(filenames=['train.json', 'test.json'])
train_records, test_records = data_dict['data']

### Transformations

In [ ]:
#export
def bernoulli(p): return int(np.random.binomial(1, p))

In [ ]:
#export
class BaseTransform:
    def __init__(self, apply_always=False, p=1.0, key='features'):
        self.apply_always = apply_always
        self.p = p
        self.key = key
    def __call__(self, x):
        if self.apply_always or bernoulli(self.p):
            return self.apply(x)
        return x
    def apply(self, x):
        raise NotImplementedError()

In [ ]:
#export
class UnconditionalTransform(BaseTransform):
    def __init__(self, **kwargs):
        super().__init__(apply_always=True)

In [ ]:
#export
class Compose(UnconditionalTransform):
    def __init__(self, transforms):
        super().__init__(apply_always=True)
        self.transforms = transforms
    def apply(self, dataset_record):
        return reduce(lambda x, f: f(x), self.transforms, dataset_record)

In [ ]:
#export
class JoinChannels(UnconditionalTransform):
    def apply(self, record):
        record['features'] = np.stack(list(record['features'].values()))
        return record

In [ ]:
#export
class Resize(UnconditionalTransform):
    def __init__(self, sz):
        super().__init__()
        self.sz = (sz, sz) if isinstance(sz, int) else sz
    def apply(self, record):
        record['features'] = cv.resize(record['features'], self.sz) 
        return record

In [ ]:
#export
class ToFloat(UnconditionalTransform):
    def apply(self, record):
        record['features'] = record['features'].astype(np.float32)
        return record

In [ ]:
#export
class SwapChannels(UnconditionalTransform):
    def __init__(self, new_order):
        super().__init__()
        self.new_order = new_order
    def apply(self, record):
        record['features'] = record['features'].transpose(*self.new_order)
        return record

In [ ]:
#export
class VerticalFlip(BaseTransform):
    def apply(self, record): 
        record['features'] = cv.flip(record['features'], 0)
        return record

In [ ]:
#export
class HorizontalFlip(BaseTransform):
    def apply(self, record): 
        record['features'] = cv.flip(record['features'], 1)
        return record

In [ ]:
#export
class PixelStatsNorm(UnconditionalTransform):
    def __init__(self, stats):
        super().__init__()
        self.stats = stats
    def apply(self, record):
        keys = [(record['id_code'], i+1, record['site']) for i in range(6)]
        mean = np.stack([self.stats[key]['mean'] for key in keys]).astype(np.float32)
        std = np.stack([self.stats[key]['std'] for key in keys]).astype(np.float32)
        mean, std = [arr.reshape(len(arr), 1, 1) for arr in (mean, std)]
        norm = (record['features'] - mean)/(std + 1e-8)
        record['features'] = norm
        return record

In [ ]:
#export
class Rescale(UnconditionalTransform):
    def __init__(self, factor=1./255):
        super().__init__()
        self.factor = factor
    def apply(self, record):
        record['features'] = record['features'] * self.factor
        return record

In [ ]:
#export
class Augmented(Dataset):
    def __init__(self, ds, tr):
        self.ds = ds
        self.tr = tr
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, index):
        return self.tr(self.ds[index])

### Composition and Normalization Stats Reading

In [ ]:
#export
stats = build_stats_index(ROOT/'pixel_stats.csv')

train_transform = Compose([
    JoinChannels(),
    SwapChannels((1, 2, 0)),
    Resize(224),
    ToFloat(),
    VerticalFlip(p=0.25),
    HorizontalFlip(p=0.25),
    SwapChannels((2, 0, 1)),
    PixelStatsNorm(stats),
    Rescale()
])

valid_transform = Compose([
    JoinChannels(),
    SwapChannels((1, 2, 0)),
    Resize(224),
    ToFloat(),
    SwapChannels((2, 0, 1)),
    PixelStatsNorm(stats),
    Rescale()
])

test_transform = Compose([
    JoinChannels(),
    SwapChannels((1, 2, 0)),
    Resize(224),
    ToFloat(),
    SwapChannels((2, 0, 1)),
    PixelStatsNorm(stats),
    Rescale()
])

In [ ]:
# list(stats)[:5]

## Pipeline

### Split

In [ ]:
#export
def split(records):
    experiments = np.unique([r['experiment'] for r in records])
    cell_types = np.unique([r['cell_type'] for r in records])
    holdouts = []
    for ct in cell_types:
        holdout = np.random.choice(experiments[
            [exp.startswith(ct) for exp in experiments]])
        holdouts.append(holdout)

    train_exp, valid_exp = [exp for exp in experiments if exp not in holdouts], holdouts
    train = [r for r in records if r['experiment'] in train_exp]
    valid = [r for r in records if r['experiment'] in valid_exp]
    
    return train, valid

### Loaders

In [ ]:
#export
def create_loaders(data, batch_size, n_cpu=12, with_test=False):
    train_records, valid_records = split(data)

    trn_ds = RxRxDataset(train_records, num_classes=NUM_CLASSES)
    aug_trn_ds = Augmented(trn_ds, train_transform)

    val_ds = RxRxDataset(valid_records, num_classes=NUM_CLASSES)
    aug_val_ds = Augmented(val_ds, valid_transform)

    loaders = OrderedDict([
        ('train', DataLoader(
            aug_trn_ds, batch_size=batch_size, 
            shuffle=True, num_workers=n_cpu)),
        ('valid', DataLoader(
            aug_val_ds, batch_size=batch_size, 
            shuffle=False, num_workers=n_cpu)),
    ])
    
    return loaders

## Model

In [ ]:
#export
from catalyst.contrib.modules import GlobalConcatPool2d

In [ ]:
#export
def create_resnet_model(size, num_classes):
    model_name = f'resnet{size}'
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    feat_dim = model.last_linear.in_features
    model.last_linear = nn.Sequential(
        nn.Linear(feat_dim, feat_dim // 2),
        nn.Dropout(0.5),
        nn.BatchNorm1d(feat_dim // 2),
        nn.ReLU(inplace=True),
        nn.Linear(feat_dim // 2, num_classes)
    )
    conv1 = model.conv1.weight
    new_conv = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)
    with torch.no_grad():
        new_conv.weight[:,:] = torch.stack([torch.mean(conv1, 1)]*6, dim=1)
    model.conv1 = new_conv
    return model

In [ ]:
#export
class RollingLoss:
    def __init__(self, smooth=0.98):
        self.smooth = smooth
        self.prev = 0
    def __call__(self, curr, batch_no):
        a = self.smooth
        avg_loss = a*self.prev + (1 - a)*curr
        debias_loss = avg_loss/(1 - a**batch_no)
        self.prev = avg_loss
        return debias_loss

In [ ]:
#export
def freeze_all(model):
    for child in model.children():
        for param in child.parameters():
            param.requires_grad = False

In [ ]:
#export
def unfreeze_all(model):
    print('Unfreezing all model layers')
    for child in model.children():
        for param in child.parameters():
            param.requires_grad = True

In [ ]:
#export
def unfreeze_layers(model, names):
    print('Unfreezing layers:', names)
    for name, child in model.named_children():
        if name in names:
            for param in child.parameters():
                param.requires_grad = True

In [ ]:
#export
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#export
device = torch.device('cuda:0')

In [ ]:
#export
model = create_resnet_model(size=101, num_classes=NUM_CLASSES)
freeze_all(model)

In [ ]:
#export
epochs = 1000
patience = 500

opt = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
sched = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.98)
model = model.to(device)
rolling_loss = dict(train=RollingLoss(), valid=RollingLoss())
steps = dict(train=0, valid=0)

trials = 0
best_metric = -np.inf
history = []
stop = False

vis = Visdom(server='0.0.0.0', port=9091,
             username=os.environ['VISDOM_USERNAME'],
             password=os.environ['VISDOM_PASSWORD'])

for epoch in range(1, epochs+1):
    print(f'Epoch [{epoch}/{epochs}]')
    curr_lr = opt.param_groups[0]['lr']
    vis.line(X=[epoch], Y=[curr_lr], win='lr', name='lr', update='append')
    
    if epoch == 1:
        unfreeze_layers(model, ['last_linear'])
        loaders = create_loaders(train_records, batch_size=400)
    elif epoch == 2:
        unfreeze_layers(model, ['layer4'])
        loaders = create_loaders(train_records, batch_size=200)
    elif epoch == 3:
        unfreeze_layers(model, ['layer3'])
        loaders = create_loaders(train_records, batch_size=40)
    elif epoch == 4:
        unfreeze_layers(model, ['layer2'])
    elif epoch == 5:
        unfreeze_all(model)
        
    iteration = dict(epoch=epoch, train_loss=list(), valid_loss=list())
    print(f"Current learning rate: {opt.param_groups[0]['lr']:.2E}")
    
    for name, loader in loaders.items():
        is_training = name == 'train'
        count = 0
        metric = 0.0
        
        with torch.set_grad_enabled(is_training):
            for batch in loader:
                steps[name] += 1
                opt.zero_grad()
                x = batch['features'].to(device)
                y = batch['targets'].to(device)
                out = model(x)
                
                if is_training:
                    loss = loss_fn(out, y)
                    loss.backward()
                    opt.step()
                
                avg_loss = rolling_loss[name](loss.item(), steps[name])
                iteration[f'{name}_loss'].append(avg_loss)
                y_pred = out.softmax(dim=1).argmax(dim=1)
                acc = (y_pred == y).float().mean().item()
                metric += acc
                count += len(batch)
                vis.line(X=[steps[name]], Y=[avg_loss], name=f'{name}_loss', 
                         win=f'{name}_loss', update='append', 
                         opts=dict(title=f'Running Loss [{name}]'))
        
        metric /= count
        iteration[f'{name}_acc'] = metric
        vis.line(X=[epoch], Y=[avg_loss], name=f'{name}', win='avg_loss',
                 update='append', opts=dict(title='Average Epoch Loss'))
        vis.line(X=[epoch], Y=[metric], name=f'{name}', win='accuracy', 
                 update='append', opts=dict(title=f'Accuracy'))
        
        last_loss = iteration[f'{name}_loss'][-1]
        
        print(f'{name} metrics: accuracy={metric:2.3%}, loss={last_loss:.4f}')
          
        if is_training:
            sched.step()
          
        else:
            # sched.step(metric)
            if metric > best_metric:
                trials = 0
                best_metric = metric
                print('Score improved!')
                torch.save(model.state_dict(), f'train.{epoch}.pth')
            else:
                trials += 1
                if trials >= patience:
                    stop = True
                    break
    
    history.append(iteration)
    
    print('-' * 80)
    
    if stop:
        print(f'Early stopping on epoch: {epoch}')
        break

torch.save(history, 'history.pth')